# Project 3

- Load (symmetric) TSP data from http://www.iwr.uni-heidelberg.de/groups/comopt/software/TSPLIB95/ 
- For opening the data use https://github.com/tsartsaris/TSPLIB-python-parser
- Apply the MMAS algorithm described in the exercise sheet

In [3]:
import itertools as it
import time
import numpy as np

In [65]:
'''
    Max-Mint Ant System for solving the traveling salesman problem
    Parameters:
        G - mapping of nodes with their coordinates
        OPT - length of the optimal route (given with the dataset)
    Optional values (given as keyword arguments):
        rho  (default: 1 / n)
        tau_min  (default: 1 / n**2)
        tau_max  (default: n - 1 / n)
        alpha  (default: 1)
        beta  (default: 0)
'''
class MMAS:
    def __init__(self, G, OPT, **kwargs):
        self.N, self.n, self.am = self._process_graph(G)
        self.OPT = OPT
        self.start_time = None
        self.wall_clock_time = 6  # For the beginning test 6 seconds - Later: 10 * 60 (10 minutes)
        # Pheromone values stored in an adjacent matrix (initial 1 / n for each edge, otherwise 0)
        self.tau = self._init_tau()
        # Parameters of the algorithm
        self.rho = kwargs.get('rho') or 1 / self.n
        self.tau_min = kwargs.get('tau_min') or 1 / self.n**2
        self.tau_max = kwargs.get('tau_max') or self.n - 1 / self.n
        self.alpha = kwargs.get('alpha') or 1
        self.beta = kwargs.get('beta') or 0
        
    def _process_graph(self, G):
        n = len(G)
        # Assuming that the ids are from 1 to n
        N = np.arange(n)
        positions = np.zeros((n, 2), dtype=np.float)
        # Collect all positions assuming that the nodes are sorted by id
        positions = [(float(x), float(y)) for x, y in G.values()]
        # Generate adjacent matrix containing the euclidean distances between nodes
        am = np.zeros((n, n), dtype=np.float)
        for i, j in it.combinations(N, 2):
            i_x, i_y = positions[i]
            j_x, j_y = positions[j]
            am[i, j] = am[j, i] = np.hypot((i_x - j_x), (i_y - j_y))
        return N, n, am
        
    def _init_tau(self):
        return np.array([[int(x != y) / self.n for x in range(self.n)] for y in range(self.n)])

    #--------------------------------------------------------------------------------------------------------------
    # Methods for execution
    #--------------------------------------------------------------------------------------------------------------
    
    def _route_length(self, x):
        # Sum up the distances between all traversed nodes
        return sum([self.am[x[i], x[i+1]] for i in range(len(x) - 1)])
    
    def _is_optimum_found(self, x):
        # Optimal value found or the wall clock time is reached
        return (self._route_length(x) == self.OPT or
                time.time() - self.start_time > self.wall_clock_time)
    
    def _update_tau(self, x):
        # TODO:
        # Update self.tau regarding the path x (list of visited nodes)
        pass
    
    def _construct(self):
        # Create a route based on the pheromone values in self.tau
        route = np.empty(self.n, dtype=np.int)
        # Setup a visited array to prevent visiting a node twice
        visited = np.zeros(self.n, dtype=np.bool)
        # Start at a random node
        route[0] = np.random.randint(self.n)
        visited[route[0]] = True
        for i in range(self.n - 1):
            v_k = route[i]
            left_nodes = np.array([(z, self.tau[v_k, z]**self.alpha / self.am[v_k, z]**self.beta)
                                   for z in self.N if not visited[z]])
            R = left_nodes[:, 1].sum()
            route[i + 1] = np.random.choice(left_nodes[:, 0], p=left_nodes[:, 1] / R)
            visited[route[i + 1]] = True
        return route
    
    def __call__(self):
        self.start_time = time.time()
        best_x = self._construct()
        self._update_tau(best_x)
        while not self._is_optimum_found(best_x):
            x = self._construct()
            if self._route_length(x) < self._route_length(best_x):
                best_x = x
            self._update_tau(best_x)
        return best_x, self._route_length(best_x)

In [66]:
# Always receive the same random decisions
np.random.seed(0)
# Sample data from the python library
sample = dict([(1, ('38.24', '20.42')), (2, ('39.57', '26.15')), (3, ('40.56', '25.32')), (4, ('36.26', '23.12')), (5, ('33.48', '10.54')), (6, ('37.56', '12.19')), (7, ('38.42', '13.11')), (8, ('37.52', '20.44')), (9, ('41.23', '9.10')), (10, ('41.17', '13.05')), (11, ('36.08', '-5.21')), (12, ('38.47', '15.13')), (13, ('38.15', '15.35')), (14, ('37.51', '15.17')), (15, ('35.49', '14.32')), (16, ('39.36', '19.56'))])
MMAS(sample, -1)()

(array([ 2,  1, 11, 14,  5,  6,  4, 12, 15,  0,  3,  7,  9,  8, 13, 10]),
 83.801096822441664)

In [12]:
list(it.combinations([enumerate(['a', 'b', 'c']), enumerate(['a', 'b', 'c'])], 2))

[((0, 'a'), (1, 'b')), ((0, 'a'), (2, 'c')), ((1, 'b'), (2, 'c'))]